## Что нужно обосновать?



**Model Dependency**

Фиксируем Датасет 

Метрика: число разных кластеров (т.е. сколько у разных моделей разных кластеров)

**Randomization**

Просто средняя дисперсия по всей кривой? Отдельно -- пересечения (Жакар?) оптимумов, и дисперсия внутри оптимумов (по всем точкам, которые попали хотя бы в один оптимум) 

**Methods' disagreement**

Кажется, тут нужно для каждого датасета и модели посчитать интервалы числа тем и дать статистику о том, насколько их пересечение пустое?


> и между априорными предположениями о числе тем/категорий, когда они есть у датасета. 

Кажется, тут нужно показать пальцем на WRef220?



**Objectivity concerns**
???



**Synthetic corpus**
???

## Properties of studied metrics 

**Diversity**

Diversity vs max(AIC): во сколько раз больше

какую диверсити взять? давай пока все, потом обсудим

каждая клеточка -- это датасет и модель

**Information-theoretic**

Модель х Датасет

смог ли он сделать оценку? через запятую

**expected results**

WRef, 20NG: 
метрика, модель, значение $T$. Болдом отметить там, где оно ОК


**TODO:** починить Outside и попробовать плато

In [1]:
import glob
import itertools
import os
import time
import sys

import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
sys.path.insert(0, '../OptimalNumberOfTopics/')     # topnum

sys.path.insert(1, '/home/bulatov/bb_topicnet/')  # topicnet

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from topicnet.cooking_machine.models import TopicModel
from topicnet.cooking_machine.dataset import Dataset

from topnum.data.vowpal_wabbit_text_collection import VowpalWabbitTextCollection
from topnum.search_methods.optimize_scores_method import OptimizeScoresMethod
from topnum.utils import (
    read_corpus_config, split_into_train_test, 
    build_every_score, monotonity_and_std_analysis, 
    trim_config, classify_curve, SCORES_DIRECTION, load_models_from_disk
)
from topnum.model_constructor import KnownModel, PARAMS_EXPLORED
from topnum.utils import estimate_num_iterations_for_convergence

from collections import defaultdict

In [5]:
from topnum.utils import magic_clutch


magic_clutch()

../OptimalNumberOfTopics/topnum/utils.py:133: UserWarning: Dataset seems to be in BOW! Skipping coherence scores
  warnings.warn('Dataset seems to be in BOW! Skipping coherence scores')


In [6]:
import topnum.scores.base_custom_score as base_custom_score

base_custom_score.__NO_LOADING_DATASET__[0] = True


In [7]:


EXPERIMENTS_DICT = {
    "20NewsGroups": "/data/_tmp_alekseev/OptNumExperiments/AllDatasets/20NG_20NG_NEW",
    # "RuWikiGood": 
    "StackOverflow": "/data/_tmp_alekseev/OptNumExperiments/AllDatasets/SO_SO_NEW",
    "WikiRef220": "/data/_tmp_alekseev/OptNumExperiments/AllDatasets/WRef_NEW/",
    "PostNauka": "/data/_tmp_alekseev/OptNumExperiments/AllDatasets/PN_PN_NEW",
    # "Reuters": "/data/_tmp_alekseev/OptNumExperiments/AllDatasets/"
    "Brown": "/data/_tmp_alekseev/OptNumExperiments/AllDatasets/Brown_Brown_NEW",
}


In [8]:
import warnings

warnings.filterwarnings("ignore", category=UserWarning)

In [9]:
# %%prun -s cumulative -q -l 600 -T prun0

EXPERIMENT_NAME_TEMPLATE = "_{mfv}_{param_id}_{seed}"

configs_dir = os.path.join('..', 'OptimalNumberOfTopics', 'topnum', 'configs')
configs_mask = os.path.join(configs_dir, '*.yml')

data_results = []
optimum_tolerance = 0.07

from time import time

start = time()
start2 = time()

for config_file in glob.glob(configs_mask):
    config = read_corpus_config(config_file)

    if config['name'] in EXPERIMENTS_DICT:
        print(config['name'])
        experiment_directory = EXPERIMENTS_DICT[config['name']]
        
        for model_family in KnownModel:
            #if model_family != KnownModel.ARTM:
            #    continue
            # print(model_family, end=", ")
            print(model_family, (time() - start)/60)
            start = time()
            tmp = "WRef_test" if config['name'] == "WikiRef220" else config['batches_prefix']
            template = tmp + EXPERIMENT_NAME_TEMPLATE.format(
                mfv=model_family.value, param_id="{}", seed="{}"
            )

            details = defaultdict(dict)

            all_subexperems_mask = os.path.join(
                experiment_directory, template.format("*", "*")
            )

            for entry in glob.glob(all_subexperems_mask):
                experiment_name = entry.split("/")[-1]

                result, detailed_result = load_models_from_disk(
                    experiment_directory, experiment_name
                )

                for score in detailed_result.keys():
                    if SCORES_DIRECTION[score] is not None:
                        details[score][experiment_name] = detailed_result[score].T
            for score in details.keys():
                for experiment_name, data in details[score].items():
                    
                    *name_base, param_id, seed = experiment_name.split("_")
                    seed = int(seed)
                    my_data = data.T.mean(axis=0)

                    score_direction = SCORES_DIRECTION[score]
                    colored_values, curve_type = classify_curve(my_data, optimum_tolerance, score_direction)
                    
                    data_results.append(
                        [
                            config['name'], model_family.value, param_id, seed, score, 
                            str(curve_type).split(".")[1], 
                            list(colored_values[colored_values.notna()].index),
                            data.values.T.tolist()[0], data.index.tolist(), 
                            config['max_num_topics'],
                            config['min_num_topics'],
                        ]
                    )
            print()

end = time()
print((end-start2)/60)


20NewsGroups
KnownModel.LDA 0.00011119842529296874


/data/topicnet/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1113: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)



KnownModel.PLSA 0.0398169199625651

KnownModel.SPARSE 0.013310523827870686

KnownModel.TLESS 0.0553516149520874

KnownModel.DECORRELATION 0.013440720240275065

KnownModel.ARTM 0.039581231276194256

StackOverflow
KnownModel.LDA 0.16148954232533771

KnownModel.PLSA 0.2627339680989583

KnownModel.SPARSE 0.06875329812367757

KnownModel.TLESS 0.3156597336133321

KnownModel.DECORRELATION 0.08593140045801799

KnownModel.ARTM 0.2322750210762024

WikiRef220
KnownModel.LDA 0.9143106460571289

KnownModel.PLSA 0.056965736548105876

KnownModel.SPARSE 0.017339551448822023

KnownModel.TLESS 0.0670362909634908

KnownModel.DECORRELATION 0.017542994022369383

KnownModel.ARTM 0.05123396714528402

PostNauka
KnownModel.LDA 0.21040994326273602

KnownModel.PLSA 0.051790507634480794

KnownModel.SPARSE 0.016724054018656412

KnownModel.TLESS 0.06679354111353557

KnownModel.DECORRELATION 0.0166743000348409

KnownModel.ARTM 0.049185311794281004

Brown
KnownModel.LDA 0.19660035769144693

KnownModel.PLSA 0.0400899

In [10]:
import pandas as pd

In [11]:
rwg_df = pd.read_pickle("rwg_df.pkl")

rwg_df

,corpus,model_family,parameters_id,seed,score,curve_type,optimums,numeric_values,T_index,max_num_topics,min_num_topics
0,RuWikiGood,LDA,0,0,SparsityThetaScore,EMPTY,"[2, 4, 8, 10, 20, 30, 40, 50, 60, 70, 80, 90, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[2, 4, 8, 10, 20, 30, 40, 50, 60, 70, 80, 90, ...",100,5
1,RuWikiGood,LDA,2,0,SparsityThetaScore,EMPTY,"[2, 4, 8, 10, 20, 30, 40, 50, 60, 70, 80, 90, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[2, 4, 8, 10, 20, 30, 40, 50, 60, 70, 80, 90, ...",100,5
2,RuWikiGood,LDA,2,2,SparsityThetaScore,EMPTY,"[2, 4, 8, 10, 20, 30, 40, 50, 60, 70, 80, 90, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[2, 4, 8, 10, 20, 30, 40, 50, 60, 70, 80, 90, ...",100,5
3,RuWikiGood,LDA,0,2,SparsityThetaScore,EMPTY,"[2, 4, 8, 10, 20, 30, 40, 50, 60, 70, 80, 90, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[2, 4, 8, 10, 20, 30, 40, 50, 60, 70, 80, 90, ...",100,5
4,RuWikiGood,LDA,2,1,SparsityThetaScore,EMPTY,"[2, 4, 8, 10, 20, 30, 40, 50, 60, 70, 80, 90, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[2, 4, 8, 10, 20, 30, 40, 50, 60, 70, 80, 90, ...",100,5
...,...,...,...,...,...,...,...,...,...,...,...
2083,RuWikiGood,ARTM,11,2,lift,JUMP_OUTSIDE,"[71, 91, 101]","[-1.4707727947468596, 4.373855301183862, 15.24...","[3, 5, 9, 11, 21, 31, 41, 51, 61, 71, 81, 91, ...",100,5
2084,RuWikiGood,ARTM,6,0,lift,OUTSIDE,"[71, 81, 91, 101]","[-12.039903371843643, -7.042359885867159, 3.58...","[3, 5, 9, 11, 21, 31, 41, 51, 61, 71, 81, 91, ...",100,5
2085,RuWikiGood,ARTM,10,1,lift,OUTSIDE,"[81, 91, 101]","[-0.7669003183301714, 1.0327192921045267, 11.0...","[3, 5, 9, 11, 21, 31, 41, 51, 61, 71, 81, 91, ...",100,5
2086,RuWikiGood,ARTM,9,0,lift,OUTSIDE,"[81, 91, 101]","[-1.8358684832815395, -3.2561314024196806, 6.0...","[3, 5, 9, 11, 21, 31, 41, 51, 61, 71, 81, 91, ...",100,5


In [12]:
df = pd.DataFrame(data=data_results, columns=["corpus", "model_family", "parameters_id", "seed", "score", "curve_type", "optimums", 'numeric_values', 'T_index', 'max_num_topics', 'min_num_topics'])

df

,corpus,model_family,parameters_id,seed,score,curve_type,optimums,numeric_values,T_index,max_num_topics,min_num_topics
0,20NewsGroups,LDA,1,2,SparsityThetaScore,EMPTY,"[2, 5, 8, 11, 14, 17, 20, 23, 26, 29, 32, 35, 38]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[2, 5, 8, 11, 14, 17, 20, 23, 26, 29, 32, 35, 38]",30,10
1,20NewsGroups,LDA,0,1,SparsityThetaScore,EMPTY,"[2, 5, 8, 11, 14, 17, 20, 23, 26, 29, 32, 35, 38]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[2, 5, 8, 11, 14, 17, 20, 23, 26, 29, 32, 35, 38]",30,10
2,20NewsGroups,LDA,2,2,SparsityThetaScore,EMPTY,"[2, 5, 8, 11, 14, 17, 20, 23, 26, 29, 32, 35, 38]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[2, 5, 8, 11, 14, 17, 20, 23, 26, 29, 32, 35, 38]",30,10
3,20NewsGroups,LDA,0,0,SparsityThetaScore,EMPTY,"[2, 5, 8, 11, 14, 17, 20, 23, 26, 29, 32, 35, 38]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[2, 5, 8, 11, 14, 17, 20, 23, 26, 29, 32, 35, 38]",30,10
4,20NewsGroups,LDA,1,1,SparsityThetaScore,EMPTY,"[2, 5, 8, 11, 14, 17, 20, 23, 26, 29, 32, 35, 38]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[2, 5, 8, 11, 14, 17, 20, 23, 26, 29, 32, 35, 38]",30,10
...,...,...,...,...,...,...,...,...,...,...,...
11011,Brown,ARTM,0,2,new_holdout_perp,OUTSIDE,"[27, 30]","[2403.397705078125, 2295.171875, 2249.53222656...","[6, 9, 12, 15, 18, 21, 24, 27, 30]",25,5
11012,Brown,ARTM,8,1,new_holdout_perp,OUTSIDE,"[27, 30]","[2883.9013671875, 2883.901123046875, 2428.6345...","[6, 9, 12, 15, 18, 21, 24, 27, 30]",25,5
11013,Brown,ARTM,7,0,new_holdout_perp,OUTSIDE,"[27, 30]","[2527.578125, 2419.661376953125, 2345.38452148...","[6, 9, 12, 15, 18, 21, 24, 27, 30]",25,5
11014,Brown,ARTM,11,2,new_holdout_perp,OUTSIDE,"[24, 27, 30]","[2883.90185546875, 2883.901611328125, 2883.901...","[6, 9, 12, 15, 18, 21, 24, 27, 30]",25,5


In [13]:
df = pd.concat([df, rwg_df])

In [14]:
# df.pivot_table(index=['score', 'curve_type'], aggfunc='count')
'''
table = df.query("model_family != 'ARTM'").pivot_table(
    values="seed", 
    index=['score'], columns=['curve_type'], aggfunc='count', 
    fill_value=0
)
'''


'\ntable = df.query("model_family != \'ARTM\'").pivot_table(\n    values="seed", \n    index=[\'score\'], columns=[\'curve_type\'], aggfunc=\'count\', \n    fill_value=0\n)\n'

In [15]:
# df.query("curve_type == 'PEAK' and score == 'toptok1'")



**Что мы видим:** у топ-токенов острый максимум, но он не совпадает по разным перезапускам

## Устойчивсть по рандомизации

Просто средняя дисперсия по всей кривой? Отдельно -- пересечения (Жакар?) оптимумов, и дисперсия внутри оптимумов (по всем точкам, которые попали хотя бы в один оптимум)

In [16]:
from functools import reduce


In [17]:
def calc_stability(gdf):

    optima_union = list(reduce(lambda a, b: set(a) | set(b), gdf.optimums))
    optima_intersect = list(reduce(lambda a, b: set(a) & set(b), gdf.optimums))


    tmp = pd.DataFrame.from_records(gdf.numeric_values.values).T


    tmp['T_index'] = gdf.T_index.iloc[0]

    tmp.set_index("T_index", inplace=True)

    scale = tmp.max().max() - tmp.min().min()
    relative_delta = (tmp.max(axis=1) - tmp.min(axis=1)) / scale
    if optima_union:
        jaccard = 1 - len(optima_intersect)/len(optima_union)
    else:
        jaccard = float("nan") 

    return relative_delta.loc[optima_union].mean(), relative_delta.mean(), jaccard



In [18]:
#groupee = df.query("curve_type != 'OUTSIDE' and curve_type != 'EMPTY'").groupby(["corpus", "model_family", 'parameters_id', 'score'])



In [19]:
groupee = df.groupby(["corpus", "model_family", 'parameters_id', 'score'])


In [20]:
calculated_stab = pd.DataFrame(groupee.apply(calc_stability))


calculated_stab["avg_rel_delta_opt"] = calculated_stab[0].apply(lambda x: x[0])
calculated_stab["avg_rel_delta_all"] = calculated_stab[0].apply(lambda x: x[1])
calculated_stab["jaccard"] = calculated_stab[0].apply(lambda x: x[2])

calculated_stab.drop(columns=[0], inplace=True)




/data/topicnet/lib/python3.6/site-packages/ipykernel/__main__.py:14: RuntimeWarning: invalid value encountered in double_scalars


In [21]:
calculated_stab.groupby(['score', 'corpus']).agg('mean')['jaccard'].unstack()#.sort_values(by=['jaccard'])

corpus,20NewsGroups,Brown,PostNauka,RuWikiGood,StackOverflow,WikiRef220
score,,,,,,
AIC_sparsity_False,0.236111,0.309028,0.276359,0.234028,0.015278,0.611111
AIC_sparsity_True,0.378472,0.319444,0.198148,0.156746,0.023611,0.240278
BIC_sparsity_False,0.175099,0.125000,0.208333,0.041667,0.015278,0.204861
BIC_sparsity_True,0.395833,0.320833,0.259722,0.287897,0.023611,0.356180
MDL_sparsity_False,0.210813,0.062500,0.111111,0.041667,0.034722,0.118056
MDL_sparsity_True,0.196181,0.383333,0.182738,0.325347,0.016667,0.588095
SparsityPhiScore@lemmatized,NaN,NaN,NaN,0.050099,0.080724,0.309375
SparsityPhiScore@word,0.194097,0.270833,0.229167,NaN,NaN,NaN
SparsityThetaScore,0.392361,0.405093,0.444444,0.148878,0.221627,0.255556


In [22]:
df.query("score == 'TopicKernel@lemmatized.average_purity' and model_family == 'PLSA' and corpus == 'RuWikiGood'")

,corpus,model_family,parameters_id,seed,score,curve_type,optimums,numeric_values,T_index,max_num_topics,min_num_topics
270,RuWikiGood,PLSA,0,2,TopicKernel@lemmatized.average_purity,PEAK,[2],"[0.9437206983566284, 0.640830397605896, 0.4214...","[2, 4, 8, 10, 20, 30, 40, 50, 60, 70, 80, 90, ...",100,5
271,RuWikiGood,PLSA,0,1,TopicKernel@lemmatized.average_purity,PEAK,[2],"[0.9266183376312256, 0.6759700775146484, 0.438...","[2, 4, 8, 10, 20, 30, 40, 50, 60, 70, 80, 90, ...",100,5
272,RuWikiGood,PLSA,0,0,TopicKernel@lemmatized.average_purity,PEAK,[2],"[0.9151639342308044, 0.6561241149902344, 0.431...","[2, 4, 8, 10, 20, 30, 40, 50, 60, 70, 80, 90, ...",100,5


In [23]:
row = df.query("curve_type == 'PEAK'").iloc[0]

row

corpus                                                 20NewsGroups
model_family                                                    LDA
parameters_id                                                     1
seed                                                              2
score                             TopicKernel@word.average_contrast
curve_type                                                     PEAK
optimums                                                        [2]
numeric_values    [0.7118598222732544, 0.6064776182174683, 0.580...
T_index           [2, 5, 8, 11, 14, 17, 20, 23, 26, 29, 32, 35, 38]
max_num_topics                                                   30
min_num_topics                                                   10
Name: 18, dtype: object

In [24]:
int_len = df.optimums.apply(len)

df[int_len < 3].query("curve_type != 'OUTSIDE' and curve_type != 'PEAK'")


,corpus,model_family,parameters_id,seed,score,curve_type,optimums,numeric_values,T_index,max_num_topics,min_num_topics
99,20NewsGroups,LDA,1,2,diversity_cosine_True,INTERVAL,"[20, 23]","[0.611115527947226, 0.5003353265755183, 0.5372...","[2, 5, 8, 11, 14, 17, 20, 23, 26, 29, 32, 35, 38]",30,10
154,20NewsGroups,LDA,0,1,renyi_0.5,INTERVAL,"[5, 8]","[1.793528558201512, 1.369764850963021, 1.39020...","[2, 5, 8, 11, 14, 17, 20, 23, 26, 29, 32, 35, 38]",30,10
161,20NewsGroups,LDA,0,2,renyi_0.5,INTERVAL,"[5, 8]","[2.207309787429051, 1.362934934886515, 1.40131...","[2, 5, 8, 11, 14, 17, 20, 23, 26, 29, 32, 35, 38]",30,10
164,20NewsGroups,LDA,2,2,renyi_1,INTERVAL,"[5, 8]","[2.9872550192704104, 1.897249471433533, 1.8436...","[2, 5, 8, 11, 14, 17, 20, 23, 26, 29, 32, 35, 38]",30,10
168,20NewsGroups,LDA,1,0,renyi_1,INTERVAL,"[14, 17]","[3.083836435631304, 3.4784739996067406, 2.7635...","[2, 5, 8, 11, 14, 17, 20, 23, 26, 29, 32, 35, 38]",30,10
...,...,...,...,...,...,...,...,...,...,...,...
2027,RuWikiGood,ARTM,6,1,uni_theta_divergence,JUMP_OUTSIDE,"[81, 101]","[0.28376902184305625, 0.2814410346031787, 0.17...","[3, 5, 9, 11, 21, 31, 41, 51, 61, 71, 81, 91, ...",100,5
2042,RuWikiGood,ARTM,7,1,uni_theta_divergence,JUMP_OUTSIDE,"[81, 101]","[0.3961730606338031, 0.4086407693878249, 0.301...","[3, 5, 9, 11, 21, 31, 41, 51, 61, 71, 81, 91, ...",100,5
2048,RuWikiGood,ARTM,6,0,uni_theta_divergence,JUMP_OUTSIDE,"[3, 101]","[0.34954422856971146, 0.1612501239137568, 0.23...","[3, 5, 9, 11, 21, 31, 41, 51, 61, 71, 81, 91, ...",100,5
2049,RuWikiGood,ARTM,10,1,uni_theta_divergence,INTERVAL,"[3, 5]","[0.7330596926129296, 0.7142295348335457, 0.399...","[3, 5, 9, 11, 21, 31, 41, 51, 61, 71, 81, 91, ...",100,5


In [25]:

def peak_hack(row):
    fixed_val = row.curve_type
    if len(row.optimums) < 3:
        if len(row.optimums) == 1:
            if row.optimums[0] in [row.T_index[0], row.T_index[-1]]:
                fixed_val = "OUTSIDE"
        if len(row.optimums) == 2:
            if set(row.optimums) == set(row.T_index[:2]):
                fixed_val = "OUTSIDE"
            if set(row.optimums) == set(row.T_index[-2:]):
                fixed_val = "OUTSIDE"
    return fixed_val


df.curve_type = df.apply(peak_hack, axis=1)

In [26]:
calculated_stab.groupby(['score', 'model_family']).agg('mean')['jaccard'].unstack()#.sort_values(by=['jaccard'])

model_family,ARTM,LDA,PLSA,TARTM,decorrelation,sparse
score,,,,,,
AIC_sparsity_False,0.291723,0.228086,0.097222,0.546429,0.299383,0.250231
AIC_sparsity_True,0.199074,0.228086,0.111111,0.598810,0.267725,0.170139
BIC_sparsity_False,0.134838,0.111111,0.055556,0.200000,0.111111,0.135169
BIC_sparsity_True,0.261641,0.111111,0.392094,0.550000,0.362302,0.268569
MDL_sparsity_False,0.089947,0.087963,0.000000,0.194444,0.129630,0.097222
MDL_sparsity_True,0.306779,0.087963,0.145833,0.408333,0.275000,0.361260
SparsityPhiScore@lemmatized,0.216845,0.000000,0.111111,0.097222,0.120370,0.087500
SparsityPhiScore@word,0.288426,0.000000,0.111111,0.208333,0.361111,0.172222
SparsityThetaScore,0.362269,0.000000,0.333333,0.277778,0.420109,0.313294


In [27]:
calculated_stab.groupby(['score', 'corpus']).agg('mean')['jaccard'].unstack().mean(axis=1).sort_values()

score
MDL_sparsity_False                         0.096478
BIC_sparsity_False                         0.128373
SparsityPhiScore@lemmatized                0.146733
holdout_perp                               0.217783
perp                                       0.217783
AIC_sparsity_True                          0.219450
new_holdout_perp                           0.228296
renyi_2                                    0.229811
SparsityPhiScore@word                      0.231366
silh                                       0.232755
TopicKernel@word.average_purity            0.256944
TopicKernel@word.average_contrast          0.270116
BIC_sparsity_True                          0.274013
calhar                                     0.276544
TopicKernel@lemmatized.average_contrast    0.278362
AIC_sparsity_False                         0.280319
MDL_sparsity_True                          0.282060
diversity_jensenshannon_False              0.302174
TopicKernel@lemmatized.average_purity      0.307407
Sparsi

In [28]:
RESULTS = pd.DataFrame()

In [29]:
RESULTS["avg_jaccard"] = calculated_stab.groupby(['score', 'corpus']).agg('mean')['jaccard'].unstack().mean(axis=1)

In [30]:
RESULTS

,avg_jaccard
score,
AIC_sparsity_False,0.280319
AIC_sparsity_True,0.219450
BIC_sparsity_False,0.128373
BIC_sparsity_True,0.274013
MDL_sparsity_False,0.096478
MDL_sparsity_True,0.282060
SparsityPhiScore@lemmatized,0.146733
SparsityPhiScore@word,0.231366
SparsityThetaScore,0.311326


In [31]:
calculated_stab.groupby(['score', 'corpus']).agg('mean')['jaccard'].unstack().max(axis=1)

score
AIC_sparsity_False                         0.611111
AIC_sparsity_True                          0.378472
BIC_sparsity_False                         0.208333
BIC_sparsity_True                          0.395833
MDL_sparsity_False                         0.210813
MDL_sparsity_True                          0.588095
SparsityPhiScore@lemmatized                0.309375
SparsityPhiScore@word                      0.270833
SparsityThetaScore                         0.444444
TopicKernel@lemmatized.average_contrast    0.404861
TopicKernel@lemmatized.average_purity      0.446528
TopicKernel@word.average_contrast          0.393308
TopicKernel@word.average_purity            0.322917
arun                                       0.574345
calhar                                     0.466601
diversity_cosine_False                     0.726389
diversity_cosine_True                      0.668578
diversity_euclidean_False                  0.924306
diversity_euclidean_True                   0.858978
divers

In [32]:
calculated_stab.groupby(['score']).agg('mean').sort_values(by=['avg_rel_delta_opt'])

,avg_rel_delta_opt,avg_rel_delta_all,jaccard
score,,,
MDL_sparsity_False,0.016706,0.014948,0.096478
SparsityPhiScore@lemmatized,0.024919,0.023689,0.146733
renyi_2,0.024940,0.037802,0.229811
BIC_sparsity_False,0.025903,0.018197,0.128373
holdout_perp,0.034231,0.038829,0.217783
perp,0.034231,0.038829,0.217783
renyi_1,0.039093,0.051596,0.356039
new_holdout_perp,0.043329,0.049772,0.228296
AIC_sparsity_True,0.046890,0.044413,0.219450


## Проверка на общую адекватность

In [33]:
adeq_table = df.groupby("score").apply(
    lambda gdf: gdf.curve_type.value_counts()
).unstack(fill_value=0)

for score in ["TopicKernel@{}.average_contrast", "TopicKernel@{}.average_purity", "SparsityPhiScore@{}"]:
    print(score)
    adeq_table.loc[score.format("ALL"), :] = adeq_table.loc[score.format("lemmatized"), :] + adeq_table.loc[score.format("word"), :]
    adeq_table.drop(score.format("word"), inplace=True)
    adeq_table.drop(score.format("lemmatized"), inplace=True)



TopicKernel@{}.average_contrast
TopicKernel@{}.average_purity
SparsityPhiScore@{}


In [34]:
adeq_table['IP'] = adeq_table['INTERVAL'] + adeq_table['PEAK']

In [35]:
adeq_table['IP'] / (adeq_table.sum(axis=1) - adeq_table['IP'])

score
AIC_sparsity_False                  0.541667
AIC_sparsity_True                   0.111111
BIC_sparsity_False                  0.444444
BIC_sparsity_True                   0.164352
MDL_sparsity_False                  0.488426
MDL_sparsity_True                   0.428241
SparsityThetaScore                  0.141204
arun                                0.143519
calhar                              0.157407
diversity_cosine_False              0.113426
diversity_cosine_True               0.148148
diversity_euclidean_False           0.250000
diversity_euclidean_True            0.243056
diversity_hellinger_False           0.062500
diversity_hellinger_True            0.245370
diversity_jensenshannon_False       0.053241
diversity_jensenshannon_True        0.194444
holdout_perp                        0.023148
intra                               0.343750
lift                                0.122685
new_holdout_perp                    0.025463
perp                                0.023148
reny

In [36]:
RESULTS["informativity"] = adeq_table['IP'] / (adeq_table.sum(axis=1) - adeq_table['IP'])

In [37]:
adeq_table.sort_values(by=["INTERVAL"], ascending=False)

,EMPTY,INTERVAL,JUMPING,JUMP_OUTSIDE,OUTSIDE,PEAK,IP
score,,,,,,,
AIC_sparsity_False,0.0,225.0,10.0,4.0,184.0,9.0,234.0
MDL_sparsity_False,0.0,211.0,0.0,13.0,208.0,0.0,211.0
BIC_sparsity_False,0.0,191.0,0.0,19.0,221.0,1.0,192.0
renyi_1,0.0,180.0,32.0,24.0,171.0,25.0,205.0
MDL_sparsity_True,0.0,169.0,14.0,2.0,231.0,16.0,185.0
renyi_0.5,0.0,158.0,46.0,29.0,138.0,61.0,219.0
renyi_2,0.0,126.0,8.0,27.0,268.0,3.0,129.0
TopicKernel@ALL.average_contrast,0.0,57.0,3.0,24.0,316.0,32.0,89.0
BIC_sparsity_True,0.0,55.0,5.0,40.0,316.0,16.0,71.0


In [38]:
df

,corpus,model_family,parameters_id,seed,score,curve_type,optimums,numeric_values,T_index,max_num_topics,min_num_topics
0,20NewsGroups,LDA,1,2,SparsityThetaScore,EMPTY,"[2, 5, 8, 11, 14, 17, 20, 23, 26, 29, 32, 35, 38]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[2, 5, 8, 11, 14, 17, 20, 23, 26, 29, 32, 35, 38]",30,10
1,20NewsGroups,LDA,0,1,SparsityThetaScore,EMPTY,"[2, 5, 8, 11, 14, 17, 20, 23, 26, 29, 32, 35, 38]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[2, 5, 8, 11, 14, 17, 20, 23, 26, 29, 32, 35, 38]",30,10
2,20NewsGroups,LDA,2,2,SparsityThetaScore,EMPTY,"[2, 5, 8, 11, 14, 17, 20, 23, 26, 29, 32, 35, 38]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[2, 5, 8, 11, 14, 17, 20, 23, 26, 29, 32, 35, 38]",30,10
3,20NewsGroups,LDA,0,0,SparsityThetaScore,EMPTY,"[2, 5, 8, 11, 14, 17, 20, 23, 26, 29, 32, 35, 38]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[2, 5, 8, 11, 14, 17, 20, 23, 26, 29, 32, 35, 38]",30,10
4,20NewsGroups,LDA,1,1,SparsityThetaScore,EMPTY,"[2, 5, 8, 11, 14, 17, 20, 23, 26, 29, 32, 35, 38]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[2, 5, 8, 11, 14, 17, 20, 23, 26, 29, 32, 35, 38]",30,10
...,...,...,...,...,...,...,...,...,...,...,...
2083,RuWikiGood,ARTM,11,2,lift,JUMP_OUTSIDE,"[71, 91, 101]","[-1.4707727947468596, 4.373855301183862, 15.24...","[3, 5, 9, 11, 21, 31, 41, 51, 61, 71, 81, 91, ...",100,5
2084,RuWikiGood,ARTM,6,0,lift,OUTSIDE,"[71, 81, 91, 101]","[-12.039903371843643, -7.042359885867159, 3.58...","[3, 5, 9, 11, 21, 31, 41, 51, 61, 71, 81, 91, ...",100,5
2085,RuWikiGood,ARTM,10,1,lift,OUTSIDE,"[81, 91, 101]","[-0.7669003183301714, 1.0327192921045267, 11.0...","[3, 5, 9, 11, 21, 31, 41, 51, 61, 71, 81, 91, ...",100,5
2086,RuWikiGood,ARTM,9,0,lift,OUTSIDE,"[81, 91, 101]","[-1.8358684832815395, -3.2561314024196806, 6.0...","[3, 5, 9, 11, 21, 31, 41, 51, 61, 71, 81, 91, ...",100,5


In [39]:
adeq_table.sum(axis=1)

score
AIC_sparsity_False                  666.0
AIC_sparsity_True                   480.0
BIC_sparsity_False                  624.0
BIC_sparsity_True                   503.0
MDL_sparsity_False                  643.0
MDL_sparsity_True                   617.0
SparsityThetaScore                  493.0
arun                                494.0
calhar                              500.0
diversity_cosine_False              481.0
diversity_cosine_True               496.0
diversity_euclidean_False           540.0
diversity_euclidean_True            537.0
diversity_hellinger_False           459.0
diversity_hellinger_True            538.0
diversity_jensenshannon_False       455.0
diversity_jensenshannon_True        516.0
holdout_perp                        442.0
intra                               387.0
lift                                485.0
new_holdout_perp                    443.0
perp                                442.0
renyi_0.5                           651.0
renyi_1                     

## Проверка на частную адекватность

In [40]:
df.corpus.unique()

array(['20NewsGroups', 'StackOverflow', 'WikiRef220', 'PostNauka',
       'Brown', 'RuWikiGood'], dtype=object)

In [41]:
EXPERIMENTS_EXPECTED_T = {
    '20NewsGroups': list(range(15,21)),
    'StackOverflow': [],
    'WikiRef220': [5],
    'PostNauka': list(range(15,31)),
    'Brown': list(range(10,21)),
    'RuWikiGood': list(range(7,14)) + list(range(80,100)),
    # 'RuWikiGood': list(range(80,100)),
    # 'Reuters': list(range(15, 50))
}


In [42]:
def calc_if_row_succeeded(row):
    corpus = row.corpus
    # print(corpus, set(EXPERIMENTS_EXPECTED_T[corpus]))
    result = bool(set(row.optimums) & set(EXPERIMENTS_EXPECTED_T[corpus]))
    result &= (row.curve_type != "EMPTY") & (row.curve_type != "OUTSIDE")
    return result

df['managed'] = df.apply(calc_if_row_succeeded, axis=1)

In [43]:
df.query("corpus == 'RuWikiGood' and managed == True").score.value_counts()

BIC_sparsity_False                         72
AIC_sparsity_False                         72
MDL_sparsity_False                         72
renyi_0.5                                  67
MDL_sparsity_True                          43
renyi_1                                    41
diversity_hellinger_True                   41
diversity_cosine_True                      26
BIC_sparsity_True                          25
diversity_jensenshannon_True               24
diversity_cosine_False                     23
arun                                       14
diversity_hellinger_False                  14
diversity_euclidean_False                  14
AIC_sparsity_True                          11
renyi_2                                    11
uni_theta_divergence                        9
diversity_jensenshannon_False               6
diversity_euclidean_True                    5
lift                                        4
new_holdout_perp                            3
holdout_perp                      

In [44]:
df.query("corpus == 'RuWikiGood' and managed == True").score.value_counts()

BIC_sparsity_False                         72
AIC_sparsity_False                         72
MDL_sparsity_False                         72
renyi_0.5                                  67
MDL_sparsity_True                          43
renyi_1                                    41
diversity_hellinger_True                   41
diversity_cosine_True                      26
BIC_sparsity_True                          25
diversity_jensenshannon_True               24
diversity_cosine_False                     23
arun                                       14
diversity_hellinger_False                  14
diversity_euclidean_False                  14
AIC_sparsity_True                          11
renyi_2                                    11
uni_theta_divergence                        9
diversity_jensenshannon_False               6
diversity_euclidean_True                    5
lift                                        4
new_holdout_perp                            3
holdout_perp                      

In [45]:
df.query("corpus == 'Reuters'")

,corpus,model_family,parameters_id,seed,score,curve_type,optimums,numeric_values,T_index,max_num_topics,min_num_topics,managed


In [58]:
groupee = df.groupby(["score", "corpus"])

res = groupee.apply(lambda gdf: gdf.managed.sum())
total = groupee.apply(lambda gdf: gdf.managed.count())

pivot_res = pd.DataFrame(res / total).unstack()

In [59]:
groupee2 = df.groupby(["score", "model_family"])

res2 = groupee2.apply(lambda gdf: gdf.managed.sum())
total2 = groupee2.apply(lambda gdf: gdf.managed.count())



In [60]:
pivot_res

0                      \
corpus                                  20NewsGroups     Brown PostNauka   
score                                                                      
AIC_sparsity_False                          0.000000  0.958333  0.416667   
AIC_sparsity_True                           0.013889  0.166667  0.041667   
BIC_sparsity_False                          0.194444  0.055556  0.541667   
BIC_sparsity_True                           0.069444  0.000000  0.138889   
MDL_sparsity_False                          0.652778  0.000000  0.138889   
MDL_sparsity_True                           0.083333  0.541667  0.000000   
SparsityPhiScore@lemmatized                      NaN       NaN       NaN   
SparsityPhiScore@word                       0.000000  0.069444  0.097222   
SparsityThetaScore                          0.027778  0.055556  0.111111   
TopicKernel@lemmatized.average_contrast          NaN       NaN       NaN   
TopicKernel@lemmatized.average_purity            NaN       NaN       NaN   
TopicKernel@word.average_contrast           0.125000  0.208333  0.166667   
TopicKernel@word.average_purity             0.125000  0.166667  0.166667   
arun                                        0.027778  0.000000  0.097222   
calhar                                      0.000000  0.000000  0.041667   
diversity_cosine_False                      0.013889  0.069444  0.013889   
diversity_cosine_True                       0.013889  0.208333  0.013889   
diversity_euclidean_False                   0.152778  0.194444  0.000000   
diversity_euclidean_True                    0.125000  0.180556  0.027778   
diversity_hellinger_False                   0.055556  0.097222  0.000000   
diversity_hellinger_True                    0.083333  0.027778  0.041667   
diversity_jensenshannon_False               0.000000  0.027778  0.000000   
diversity_jensenshannon_True                0.000000  0.013889  0.027778   
holdout_perp                                0.027778  0.000000  0.000000   
intra                                       0.055556  0.180556  0.166667   
lift                                        0.013889  0.069444  0.027778   
new_holdout_perp                            0.055556  0.000000  0.000000   
perp                                        0.027778  0.000000  0.000000   
renyi_0.5                                   0.152778  0.305556  0.555556   
renyi_1                                     0.625000  0.083333  0.694444   
renyi_2                                     0.388889  0.041667  0.333333   
silh                                        0.000000  0.013889  0.013889   
toptok1                                     0.152778  0.194444  0.250000   
uni_theta_divergence                        0.013889  0.055556  0.027778   

                                                                             
corpus                                  RuWikiGood StackOverflow WikiRef220  
score                                                                        
AIC_sparsity_False                        1.000000           0.0   0.513889  
AIC_sparsity_True                         0.152778           0.0   0.125000  
BIC_sparsity_False                        1.000000           0.0   0.513889  
BIC_sparsity_True                         0.347222           0.0   0.083333  
MDL_sparsity_False                        1.000000           0.0   0.277778  
MDL_sparsity_True                         0.597222           0.0   0.055556  
SparsityPhiScore@lemmatized               0.013889           0.0   0.000000  
SparsityPhiScore@word                          NaN           NaN        NaN  
SparsityThetaScore                        0.027778           0.0   0.000000  
TopicKernel@lemmatized.average_contrast   0.013889           0.0   0.125000  
TopicKernel@lemmatized.average_purity     0.000000           0.0   0.097222  
TopicKernel@word.average_contrast              NaN           NaN        NaN  
TopicKernel@word.average_purity                NaN           NaN    

In [61]:
res2['toptok1']

model_family
ARTM             40
LDA               5
PLSA              2
TARTM             0
decorrelation     6
sparse            7
dtype: int64

In [62]:
total2

score                 model_family 
AIC_sparsity_False    ARTM             216
                      LDA               54
                      PLSA              18
                      TARTM             18
                      decorrelation     54
                                      ... 
uni_theta_divergence  LDA               54
                      PLSA              18
                      TARTM             18
                      decorrelation     54
                      sparse            72
Length: 204, dtype: int64

In [65]:
pivot_res2 = pd.DataFrame(res2 / total2).unstack()

In [66]:
pivot_res2

0                      \
model_family                                 ARTM       LDA      PLSA   
score                                                                   
AIC_sparsity_False                       0.453704  0.555556  0.500000   
AIC_sparsity_True                        0.009259  0.555556  0.000000   
BIC_sparsity_False                       0.393519  0.351852  0.388889   
BIC_sparsity_True                        0.009259  0.351852  0.277778   
MDL_sparsity_False                       0.388889  0.277778  0.333333   
MDL_sparsity_True                        0.208333  0.277778  0.166667   
SparsityPhiScore@lemmatized              0.000000  0.000000  0.000000   
SparsityPhiScore@word                    0.111111  0.000000  0.000000   
SparsityThetaScore                       0.050926  0.000000  0.000000   
TopicKernel@lemmatized.average_contrast  0.064815  0.000000  0.000000   
TopicKernel@lemmatized.average_purity    0.055556  0.000000  0.000000   
TopicKernel@word.average_contrast        0.305556  0.000000  0.000000   
TopicKernel@word.average_purity          0.277778  0.000000  0.000000   
arun                                     0.060185  0.074074  0.111111   
calhar                                   0.009259  0.000000  0.000000   
diversity_cosine_False                   0.078704  0.018519  0.000000   
diversity_cosine_True                    0.120370  0.222222  0.166667   
diversity_euclidean_False                0.175926  0.018519  0.000000   
diversity_euclidean_True                 0.125000  0.000000  0.000000   
diversity_hellinger_False                0.120370  0.018519  0.000000   
diversity_hellinger_True                 0.189815  0.203704  0.111111   
diversity_jensenshannon_False            0.032407  0.018519  0.000000   
diversity_jensenshannon_True             0.027778  0.166667  0.166667   
holdout_perp                             0.013889  0.018519  0.000000   
intra                                    0.229167  0.000000  0.000000   
lift                                     0.032407  0.000000  0.000000   
new_holdout_perp                         0.023148  0.018519  0.000000   
perp                                     0.013889  0.018519  0.000000   
renyi_0.5                                0.407407  0.296296  0.222222   
renyi_1                                  0.300926  0.296296  0.388889   
renyi_2                                  0.120370  0.148148  0.111111   
silh                                     0.000000  0.092593  0.055556   
toptok1                                  0.277778  0.138889  0.166667   
uni_theta_divergence                     0.032407  0.037037  0.055556   

                                                                           
model_family                                TARTM decorrelation    sparse  
score                                                                      
AIC_sparsity_False                       0.444444      0.388889  0.583333  
AIC_sparsity_True                        0.166667      0.000000  0.013889  
BIC_sparsity_False                       0.166667      0.296296  0.500000  
BIC_sparsity_True                        0.277778      0.259259  0.013889  
MDL_sparsity_False                       0.166667      0.203704  0.416667  
MDL_sparsity_True                        0.222222      0.166667  0.222222  
SparsityPhiScore@lemmatized              0.000000      0.000000  0.027778  
SparsityPhiScore@word                    0.000000      0.000000  0.000000  
SparsityThetaScore                       0.000000      0.037037  0.041667  
TopicKernel@lemmatized.average_contrast  0.000000      0.074074  0.027778  
TopicKernel@lemmatized.average_purity    0.000000      0.037037  0.000000  
TopicKernel@word.average_contrast        0.000000      0.111111  0.000000  
TopicKernel@word.average_purity          0.000000      0.111111  0.000000  
arun                                     0.166667      0.074074  0.055556  
calhar                                   0.000000     

In [67]:
RESULTS["expected"] = pivot_res.drop(columns=[(0, "StackOverflow")]).mean(axis=1)


In [72]:
pivot_res2.sort_values(by=[(0,'PLSA')], ascending=False)
#pivot_res2.sort_values(by=['avg'], ascending=False)

0                      \
model_family                                 ARTM       LDA      PLSA   
score                                                                   
AIC_sparsity_False                       0.453704  0.555556  0.500000   
BIC_sparsity_False                       0.393519  0.351852  0.388889   
renyi_1                                  0.300926  0.296296  0.388889   
MDL_sparsity_False                       0.388889  0.277778  0.333333   
BIC_sparsity_True                        0.009259  0.351852  0.277778   
renyi_0.5                                0.407407  0.296296  0.222222   
diversity_cosine_True                    0.120370  0.222222  0.166667   
MDL_sparsity_True                        0.208333  0.277778  0.166667   
toptok1                                  0.277778  0.138889  0.166667   
diversity_jensenshannon_True             0.027778  0.166667  0.166667   
renyi_2                                  0.120370  0.148148  0.111111   
diversity_hellinger_True                 0.189815  0.203704  0.111111   
arun                                     0.060185  0.074074  0.111111   
silh                                     0.000000  0.092593  0.055556   
uni_theta_divergence                     0.032407  0.037037  0.055556   
diversity_cosine_False                   0.078704  0.018519  0.000000   
lift                                     0.032407  0.000000  0.000000   
SparsityPhiScore@lemmatized              0.000000  0.000000  0.000000   
SparsityPhiScore@word                    0.111111  0.000000  0.000000   
SparsityThetaScore                       0.050926  0.000000  0.000000   
TopicKernel@lemmatized.average_contrast  0.064815  0.000000  0.000000   
TopicKernel@lemmatized.average_purity    0.055556  0.000000  0.000000   
perp                                     0.013889  0.018519  0.000000   
new_holdout_perp                         0.023148  0.018519  0.000000   
intra                                    0.229167  0.000000  0.000000   
calhar                                   0.009259  0.000000  0.000000   
holdout_perp                             0.013889  0.018519  0.000000   
TopicKernel@word.average_contrast        0.305556  0.000000  0.000000   
diversity_jensenshannon_False            0.032407  0.018519  0.000000   
TopicKernel@word.average_purity          0.277778  0.000000  0.000000   
diversity_hellinger_False                0.120370  0.018519  0.000000   
diversity_euclidean_True                 0.125000  0.000000  0.000000   
AIC_sparsity_True                        0.009259  0.555556  0.000000   
diversity_euclidean_False                0.175926  0.018519  0.000000   

                                                                           
model_family                                TARTM decorrelation    sparse  
score                                                                      
AIC_sparsity_False                       0.444444      0.388889  0.583333  
BIC_sparsity_False                       0.166667      0.296296  0.500000  
renyi_1                                  0.222222      0.277778  0.486111  
MDL_sparsity_False                       0.166667      0.203704  0.416667  
BIC_sparsity_True                        0.277778      0.259259  0.013889  
renyi_0.5                                0.333333      0.333333  0.291667  
diversity_cosine_True                    0.111111      0.055556  0.222222  
MDL_sparsity_True                        0.222222      0.166667  0.222222  
toptok1                                  0.000000      0.166667  0.145833  
diversity_jensenshannon_True             0.055556      0.018519  0.125000  
renyi_2                                  0.333333      0.055556  0.291667  
diversity_hellinger_True                 0.000000      0.074074  0.138889  
arun                                     0.166667      0.074074  0.055556  
silh                                     0.000000      0.055556  0.013889  
uni_theta_divergence                     0.000000     

In [69]:
(pd.DataFrame(res).unstack().sum(axis=1) / pd.DataFrame(total).unstack().sum(axis=1)).sort_values(ascending=False)

score
AIC_sparsity_False                         0.481481
BIC_sparsity_False                         0.384259
renyi_0.5                                  0.354167
MDL_sparsity_False                         0.344907
renyi_1                                    0.328704
MDL_sparsity_True                          0.212963
toptok1                                    0.208333
TopicKernel@word.average_contrast          0.166667
diversity_hellinger_True                   0.157407
renyi_2                                    0.152778
TopicKernel@word.average_purity            0.152778
diversity_cosine_True                      0.143519
intra                                      0.118056
BIC_sparsity_True                          0.106481
diversity_euclidean_False                  0.099537
AIC_sparsity_True                          0.083333
diversity_euclidean_True                   0.076389
diversity_hellinger_False                  0.074074
diversity_cosine_False                     0.074074
arun  

In [70]:
df.query("managed == True and score == 'toptok1' and model_family != 'ARTM'")

,corpus,model_family,parameters_id,seed,score,curve_type,optimums,numeric_values,T_index,max_num_topics,min_num_topics,managed
697,20NewsGroups,sparse,2,0,toptok1,JUMPING,"[9, 15]","[1.3479616704909092, 1.2873354601112474, 1.352...","[3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39]",30,10,True
1080,20NewsGroups,decorrelation,2,2,toptok1,PEAK,[17],"[0.6251824956548084, 0.6251824956548084, 0.625...","[2, 5, 8, 11, 14, 17, 20, 23, 26, 29, 32, 35, 38]",30,10,True
1085,20NewsGroups,decorrelation,1,2,toptok1,PEAK,[17],"[0.6251824956548084, 1.3222306435707092, 1.372...","[2, 5, 8, 11, 14, 17, 20, 23, 26, 29, 32, 35, 38]",30,10,True
1088,20NewsGroups,decorrelation,2,1,toptok1,PEAK,[17],"[0.6251824956548084, 0.6251824956548084, 0.625...","[2, 5, 8, 11, 14, 17, 20, 23, 26, 29, 32, 35, 38]",30,10,True
4563,WikiRef220,LDA,2,0,toptok1,PEAK,[5],"[1.5485347459619556, 1.7958904636383768, 1.773...","[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1...",20,2,True
4565,WikiRef220,LDA,2,2,toptok1,PEAK,[5],"[1.754991055054276, 1.5740339492048268, 1.9463...","[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1...",20,2,True
4566,WikiRef220,LDA,1,0,toptok1,PEAK,[5],"[1.649709780215561, 1.7958904636383768, 1.9317...","[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1...",20,2,True
4570,WikiRef220,LDA,2,1,toptok1,PEAK,[5],"[1.8387568382716042, 1.4594801007343952, 2.236...","[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1...",20,2,True
4571,WikiRef220,LDA,0,1,toptok1,INTERVAL,"[4, 5]","[1.8387568382716042, 1.6979049521053802, 2.136...","[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1...",20,2,True
4680,WikiRef220,PLSA,0,1,toptok1,JUMPING,"[4, 5, 9]","[1.8387568382716042, 2.0243894425328754, 2.170...","[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1...",20,2,True


In [73]:
RESULTS

,avg_jaccard,informativity,expected
score,,,
AIC_sparsity_False,0.280319,0.541667,0.577778
AIC_sparsity_True,0.219450,0.111111,0.100000
BIC_sparsity_False,0.128373,0.444444,0.461111
BIC_sparsity_True,0.274013,0.164352,0.127778
MDL_sparsity_False,0.096478,0.488426,0.413889
MDL_sparsity_True,0.282060,0.428241,0.255556
SparsityPhiScore@lemmatized,0.146733,NaN,0.006944
SparsityPhiScore@word,0.231366,NaN,0.055556
SparsityThetaScore,0.311326,0.141204,0.044444


In [75]:
FILTERED_RESULTS = RESULTS.copy()

In [76]:
FILTERED_RESULTS.index

Index(['AIC_sparsity_False', 'AIC_sparsity_True', 'BIC_sparsity_False',
       'BIC_sparsity_True', 'MDL_sparsity_False', 'MDL_sparsity_True',
       'SparsityPhiScore@lemmatized', 'SparsityPhiScore@word',
       'SparsityThetaScore', 'TopicKernel@lemmatized.average_contrast',
       'TopicKernel@lemmatized.average_purity',
       'TopicKernel@word.average_contrast', 'TopicKernel@word.average_purity',
       'arun', 'calhar', 'diversity_cosine_False', 'diversity_cosine_True',
       'diversity_euclidean_False', 'diversity_euclidean_True',
       'diversity_hellinger_False', 'diversity_hellinger_True',
       'diversity_jensenshannon_False', 'diversity_jensenshannon_True',
       'holdout_perp', 'intra', 'lift', 'new_holdout_perp', 'perp',
       'renyi_0.5', 'renyi_1', 'renyi_2', 'silh', 'toptok1',
       'uni_theta_divergence'],
      dtype='object', name='score')

In [77]:
to_remain = ['AIC_sparsity_False', 'AIC_sparsity_True', 'BIC_sparsity_False',
       'BIC_sparsity_True', 'MDL_sparsity_False', 'MDL_sparsity_True',
       'arun', 'calhar', 'diversity_cosine_False', 'diversity_cosine_True',
       'diversity_euclidean_False', 'diversity_euclidean_True',
       'diversity_hellinger_False', 'diversity_hellinger_True',
       'diversity_jensenshannon_False', 'diversity_jensenshannon_True',
       'lift', 'new_holdout_perp', 'perp',
       'renyi_0.5', 'renyi_1', 'renyi_2', 'silh', 'toptok1',
       'uni_theta_divergence']
FILTERED_RESULTS = FILTERED_RESULTS.loc[to_remain]

In [78]:
['AIC_sparsity_False', 'AIC_sparsity_True', 'BIC_sparsity_False',
       'BIC_sparsity_True', 'MDL_sparsity_False', 'MDL_sparsity_True',
       'arun', 'calhar', 'diversity_cosine_False', 'diversity_cosine_True',
       'diversity_euclidean_False', 'diversity_euclidean_True',
       'diversity_hellinger_False', 'diversity_hellinger_True',
       'diversity_jensenshannon_False', 'diversity_jensenshannon_True',
       'lift', 'new_holdout_perp', 'perp',
       'renyi_0.5', 'renyi_1', 'renyi_2', 'silh', 'toptok1',
       'uni_theta_divergence']

renamer = {}
for elem in to_remain:
    renamed_elem = None
    if "_sparsity_" in elem:
        parts = elem.split("_")
        if parts[-1] == "False":
            renamed_elem = parts[0]
        else:
            renamed_elem = "sparse " + parts[0]

    if "diversity_" in elem:
        parts = elem.split("_")
        t = {"cosine": "COS", "euclidean": "L2", "hellinger": "H", "jensenshannon": "JH"}
        if parts[-1] == "False":
            renamed_elem = f"D-" + "avg-" + t[parts[1]]
        else:
            renamed_elem = f"D-" + "cls-" + t[parts[1]]
    if elem == 'new_holdout_perp':
        renamed_elem = "holdout_perplexity"
    if elem == 'perp':
        renamed_elem = "perplexity"
    if "renyi" in elem or "lift" in elem or "uni_theta_divergence" in elem:
        renamed_elem = elem
    if elem == 'arun':
        renamed_elem = "D-Spectral"
    if elem == 'calhar':
        renamed_elem = "CHI"
    if elem == 'silh':
        renamed_elem = "SilhC"
    if elem == 'toptok1':
        renamed_elem = "average coherence"
         

    renamer[elem] = renamed_elem.replace("_", "-")
    



In [79]:

FILTERED_RESULTS.rename(index=renamer)


,avg_jaccard,informativity,expected
score,,,
AIC,0.280319,0.541667,0.577778
sparse AIC,0.219450,0.111111,0.100000
BIC,0.128373,0.444444,0.461111
sparse BIC,0.274013,0.164352,0.127778
MDL,0.096478,0.488426,0.413889
sparse MDL,0.282060,0.428241,0.255556
D-Spectral,0.455632,0.143519,0.083333
CHI,0.276544,0.157407,0.008333
D-avg-COS,0.429921,0.113426,0.088889


In [80]:
FILTERED_RESULTS.rename(index=renamer, inplace=True)


In [81]:
print(FILTERED_RESULTS.to_latex(float_format="%.3f"))

\begin{tabular}{lrrr}
\toprule
{} &  avg\_jaccard &  informativity &  expected \\
score                &              &                &           \\
\midrule
AIC                  &        0.280 &          0.542 &     0.578 \\
sparse AIC           &        0.219 &          0.111 &     0.100 \\
BIC                  &        0.128 &          0.444 &     0.461 \\
sparse BIC           &        0.274 &          0.164 &     0.128 \\
MDL                  &        0.096 &          0.488 &     0.414 \\
sparse MDL           &        0.282 &          0.428 &     0.256 \\
D-Spectral           &        0.456 &          0.144 &     0.083 \\
CHI                  &        0.277 &          0.157 &     0.008 \\
D-avg-COS            &        0.430 &          0.113 &     0.089 \\
D-cls-COS            &        0.526 &          0.148 &     0.172 \\
D-avg-L2             &        0.682 &          0.250 &     0.119 \\
D-cls-L2             &        0.584 &          0.243 &     0.092 \\
D-avg-H              &   

In [82]:
FILTERED_RESULTS.sort_values(by="avg_jaccard").head(7)

,avg_jaccard,informativity,expected
score,,,
MDL,0.096478,0.488426,0.413889
BIC,0.128373,0.444444,0.461111
perplexity,0.217783,0.023148,0.013889
sparse AIC,0.219450,0.111111,0.100000
holdout-perplexity,0.228296,0.025463,0.019444
renyi-2,0.229811,0.298611,0.183333
SilhC,0.232755,0.078704,0.027778


In [83]:
FILTERED_RESULTS.sort_values(by="informativity").tail(7)

,avg_jaccard,informativity,expected
score,,,
sparse MDL,0.282060,0.428241,0.255556
BIC,0.128373,0.444444,0.461111
average coherence,0.779514,0.472222,0.208333
renyi-1,0.356039,0.474537,0.394444
MDL,0.096478,0.488426,0.413889
renyi-0.5,0.469522,0.506944,0.425000
AIC,0.280319,0.541667,0.577778


In [84]:
FILTERED_RESULTS.sort_values(by="expected").tail(11)

,avg_jaccard,informativity,expected
score,,,
sparse BIC,0.274013,0.164352,0.127778
D-cls-COS,0.526467,0.148148,0.172222
renyi-2,0.229811,0.298611,0.183333
D-cls-H,0.594895,0.245370,0.188889
average coherence,0.779514,0.472222,0.208333
sparse MDL,0.282060,0.428241,0.255556
renyi-1,0.356039,0.474537,0.394444
MDL,0.096478,0.488426,0.413889
renyi-0.5,0.469522,0.506944,0.425000


In [85]:
FILTERED_RESULTS.shape

(25, 3)

## разбирательство с Рейтерс (WIP)

In [ ]:
!ls -lh /data/_tmp_alekseev/OptNumExperiments/AllDatasets/Reuters_Reuters_NEW


In [ ]:
from collections import Counter

EXPERIMENT_NAME_TEMPLATE = "_{mfv}_{param_id}_{seed}"

configs_dir = os.path.join('..', 'OptimalNumberOfTopics', 'topnum', 'configs')
configs_mask = os.path.join(configs_dir, '*.yml')

for config_file in glob.glob(configs_mask):
    config = read_corpus_config(config_file)
    if config['name'] == "Reuters":
        break

    
for model_family in KnownModel:
    tmp = "WRef_test" if config['name'] == "WikiRef220" else config['batches_prefix']
    template = tmp + EXPERIMENT_NAME_TEMPLATE.format(
                mfv=model_family.value, param_id="{}", seed="{}"
    )
    experiment_directory = '/data/_tmp_alekseev/OptNumExperiments/AllDatasets/Reuters_Reuters_NEW'

    details = defaultdict(dict)

    all_subexperems_mask = os.path.join(
        experiment_directory, template.format("*", "*")
    )
    print(all_subexperems_mask)

    for entry in glob.glob(all_subexperems_mask):
        print(entry)
        experiment_name = entry.split("/")[-1]
        print(experiment_name)

        masks = [
            f"{experiment_directory}/{experiment_name}_*",
            f"{experiment_directory}/{experiment_name}/*"
        ]
        for new_exp_format, mask in enumerate(masks):
            if not len(glob.glob(mask)):
                continue
            print(experiment_name, len(glob.glob(mask)))
            cnt = Counter()
            print(
                Counter(
                    os.stat(folder).st_mode
                    for folder in glob.glob(mask)
                )
            )
            for folder2 in glob.glob(mask):
                # if os.stat(folder2).st_mode == 17901:
                if os.stat(folder2).st_mode == 16893:
                    dm = DummyTopicModel.load(folder2)
                    break
            break
        break


        #result, detailed_result = load_models_from_disk(
        #    experiment_directory, experiment_name
        #)


In [ ]:
folder, os.stat(folder).st_mode

In [ ]:
test_dataset = Dataset(
    '/home/alekseev/topicnet/tests/test_data/test_dataset.csv',
    internals_folder_path="./DELETE_ME_PLZ"
)

_ = build_every_score(test_dataset, test_dataset, config)

In [ ]:
from topnum.scores import (
        IntratextCoherenceScore, SophisticatedTopTokensCoherenceScore
)

IntratextCoherenceScore("jbi", test_dataset)
SophisticatedTopTokensCoherenceScore("sds", test_dataset)

In [ ]:
tm = TopicModel.load(folder)

In [ ]:
tm._model.num_phi_updates

In [ ]:
tm2 = TopicModel.load(folder2)

In [ ]:
tm2._model.num_phi_updates

In [ ]:
! cp /home/alekseev/OptimalNumberOfTopics/demos/*_train.csv  .

In [ ]:
for folder in glob.glob(mask):
    print(os.path.basename(folder), os.stat(folder).st_mode)


In [ ]:
experiment_directory

In [ ]:
config

In [ ]:
experiment_directory, experiment_name

In [ ]:
base_experiment_name = experiment_name